In [1]:
%%time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import fitz
from io import BytesIO
import re

url = 'https://www.zaragoza.es/sede/portal/normativa/'
ciudad = 'Zaragoza'
date = datetime.today().strftime('%Y-%m-%d')

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
links = {
    'https://www.zaragoza.es' + a_tag['href'] if a_tag['href'].startswith('/') else a_tag['href']: a_tag
    for a_tag in soup.find_all('a', href=True)
}
filtered_links = {
    href: group for href, group in links.items()
    if ('/sede/servicio/normativa/' in href or '/contenidos/normativa/ordenanzas-fiscales/' in href)
    and not (href.endswith('/sede/servicio/normativa/') or href.endswith('/contenidos/normativa/ordenanzas-fiscales/'))
}

filtered_urls = []
for href, group in filtered_links.items():
    response = requests.get(href)
    content = ""
    if href.endswith('.pdf'):
        pdf_data = BytesIO(response.content)
        doc = fitz.open("pdf", pdf_data)
        for page in doc:
            content += page.get_text()
    else:
        soup = BeautifulSoup(response.content, 'html.parser')
        content += soup.get_text()
    title = group.get_text(strip=True)
    if title == 'Acceso':
        title = group.get('data-original-title', group)
    title = re.sub(r'^Acceso\s*', '', title)
    filtered_urls.append({
        'ciudad': ciudad,
        'date': date,
        'grupo': '',
        'subgrupo': '',
        'titulo': title,
        'url': href,
        'content': content
    })

df = pd.DataFrame(filtered_urls)
df


KeyboardInterrupt

